<a href="https://colab.research.google.com/github/Abhijitgayen/SNA_As_Msc_projects/blob/main/dataSet_2017_Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Classification using LSTM and Pretrained Word2Vec

We will train and test sentence classification using LSTM, and Pretrained Word2Vec.  
You can find visualization of our code below.

###Add libary

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [3]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

### Some functions

In [4]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['tweet']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    if sentiment == 'neutral':
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['tweet'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['label'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

# Preprocess (encode text to number)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
tweetsDatas=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2016dev-A.txt', sep="\t", header=None, names=["id","label", "tweet"]); #skiping these two rows as they have some bad data
testData=pd.read_csv('/content/drive/MyDrive/2017_data_set/dataset/train/twitter-2016devtest-A.txt',sep="\t", header=None, names=["id","label", "tweet"]); #test_data
print(tweetsDatas.head())
print(tweetsDatas.label.value_counts())
print(testData.head())
print(testData.label.value_counts())

                   id     label  \
0  638060586258038784   neutral   
1  638061181823922176  positive   
2  638083821364244480   neutral   
3  638091450132078593  positive   
4  638125563790557184  positive   

                                               tweet  
0  05 Beat it - Michael Jackson - Thriller (25th ...  
1  Jay Z joins Instagram with nostalgic tribute t...  
2  Michael Jackson: Bad 25th Anniversary Edition ...  
3  I liked a @YouTube video http://t.co/AaR3pjp2P...  
4  18th anniv of Princess Diana's death. I still ...  
positive    829
neutral     746
negative    391
Name: label, dtype: int64
                   id     label  \
0  637641175948763136   neutral   
1  637651487762554881   neutral   
2  637666734300905472  negative   
3  637668142110654468   neutral   
4  637708370129125377  positive   

                                               tweet  
0  @SeeMonterey LOST - Sony cell phone with holid...  
1  @PersonaSoda well yeah, that's third parties. ...  
2  Sony r

In [6]:
df=tweetsDatas
# max_length is used for max sequence of input
max_length = get_max_length(df)
print(max_length)

train_X, train_Y = preprocess(df)

In [8]:
print(train_X)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.04849368  0.02253552 -0.03001837 ...  0.01148417 -0.00711913
    0.07169744]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
 

# Build Model

In [10]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [9]:
train_Y

array([[0.5, 0.5],
       [1. , 0. ],
       [0.5, 0.5],
       ...,
       [0.5, 0.5],
       [1. , 0. ],
       [1. , 0. ]])

In [11]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
62/62 [==============================] - 20s 272ms/step - loss: 0.6523 - accuracy: 0.7925
Epoch 2/50
62/62 [==============================] - 17s 275ms/step - loss: 0.6141 - accuracy: 0.7375
Epoch 3/50
62/62 [==============================] - 18s 286ms/step - loss: 0.5941 - accuracy: 0.7533
Epoch 4/50
62/62 [==============================] - 17s 276ms/step - loss: 0.5820 - accuracy: 0.7604
Epoch 5/50
62/62 [==============================] - 17s 274ms/step - loss: 0.5721 - accuracy: 0.7569
Epoch 6/50
62/62 [==============================] - 17s 277ms/step - loss: 0.5651 - accuracy: 0.7518
Epoch 7/50
62/62 [==============================] - 17s 270ms/step - loss: 0.5628 - accuracy: 0.7569
Epoch 8/50
62/62 [==============================] - 17s 274ms/step - loss: 0.5630 - accuracy: 0.7528
Epoch 9/50
62/62 [==============================] - 17s 274ms/step - loss: 0.5535 - accuracy: 0.7538
Epoch 10/50
62/62 [==============================] - 17s 274ms/step - loss: 0.5501

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                36224     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [15]:
test_df=testData;
print(test_df.label.value_counts())

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

positive    994
neutral     681
negative    325
Name: label, dtype: int64
63/63 - 6s - loss: 0.9267 - accuracy: 0.6850 - 6s/epoch - 89ms/step
Test score: 0.9267452955245972
Test accuracy: 0.6850000023841858


In [ ]:
test_df=testData
print(test_df.lebal.value_counts())

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)